I used the loans dataset to predict if someone will pay their loan using loan status as my dependant variable and other (numerical)
columns as my other variables. I decided to work with this database because it has a lot of numerical columns which can help determine
someones income and employment time.

In [85]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from statsmodels.formula.api import logit


df = pd.read_csv("loans_full_schema.csv")
df.head()

,emp_title,emp_length,state,homeownership,annual_income,verified_income,debt_to_income,annual_income_joint,verification_income_joint,debt_to_income_joint,...,sub_grade,issue_month,loan_status,initial_listing_status,disbursement_method,balance,paid_total,paid_principal,paid_interest,paid_late_fees
0,global config engineer,3.0,NJ,MORTGAGE,90000.0,Verified,18.01,NaN,NaN,NaN,...,C3,Mar-2018,Current,whole,Cash,27015.86,1999.33,984.14,1015.19,0.0
1,warehouse office clerk,10.0,HI,RENT,40000.0,Not Verified,5.04,NaN,NaN,NaN,...,C1,Feb-2018,Current,whole,Cash,4651.37,499.12,348.63,150.49,0.0
2,assembly,3.0,WI,RENT,40000.0,Source Verified,21.15,NaN,NaN,NaN,...,D1,Feb-2018,Current,fractional,Cash,1824.63,281.80,175.37,106.43,0.0
3,customer service,1.0,PA,RENT,30000.0,Not Verified,10.16,NaN,NaN,NaN,...,A3,Jan-2018,Current,whole,Cash,18853.26,3312.89,2746.74,566.15,0.0
4,security supervisor,10.0,CA,RENT,35000.0,Verified,57.96,57000.0,Verified,37.66,...,C3,Mar-2018,Current,whole,Cash,21430.15,2324.65,1569.85,754.80,0.0


In [86]:
df["loan_status_new"]=df['loan_status'].apply(lambda x: 1 if x in ['Late (16-30 days)','Late (31-120 days)'] else 0)


non_numeric = df.select_dtypes(include=['object']).columns
df = df.drop(non_numeric, axis=1)

df.head(485)

,emp_length,annual_income,debt_to_income,annual_income_joint,debt_to_income_joint,delinq_2y,months_since_last_delinq,earliest_credit_line,inquiries_last_12m,total_credit_lines,...,loan_amount,term,interest_rate,installment,balance,paid_total,paid_principal,paid_interest,paid_late_fees,loan_status_new
0,3.0,90000.0,18.01,NaN,NaN,0,38.0,2001,6,28,...,28000,60,14.07,652.53,27015.86,1999.33,984.14,1015.19,0.0,0
1,10.0,40000.0,5.04,NaN,NaN,0,NaN,1996,1,30,...,5000,36,12.61,167.54,4651.37,499.12,348.63,150.49,0.0,0
2,3.0,40000.0,21.15,NaN,NaN,0,28.0,2006,4,31,...,2000,36,17.09,71.40,1824.63,281.80,175.37,106.43,0.0,0
3,1.0,30000.0,10.16,NaN,NaN,0,NaN,2007,0,4,...,21600,36,6.72,664.19,18853.26,3312.89,2746.74,566.15,0.0,0
4,10.0,35000.0,57.96,57000.0,37.66,0,NaN,2008,7,22,...,23000,36,14.07,786.87,21430.15,2324.65,1569.85,754.80,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,5.0,48000.0,44.35,100000.0,35.57,0,38.0,2000,2,21,...,25875,60,24.85,757.20,24961.62,3082.39,913.38,2169.01,0.0,0
481,2.0,45000.0,16.11,NaN,NaN,0,NaN,2008,2,7,...,6000,36,15.04,208.11,4879.78,1319.32,1120.22,199.10,0.0,0
482,3.0,90000.0,10.15,109000.0,25.11,0,25.0,2003,1,20,...,40000,60,21.45,1092.29,38044.89,5413.78,1955.11,3458.67,0.0,0
483,6.0,55000.0,23.35,NaN,NaN,1,7.0,2003,1,42,...,2500,36,17.09,89.25,2224.02,441.50,275.98,165.52,0.0,0


In [87]:
df = df.fillna(df.mean())

In [88]:
df.head()

,emp_length,annual_income,debt_to_income,annual_income_joint,debt_to_income_joint,delinq_2y,months_since_last_delinq,earliest_credit_line,inquiries_last_12m,total_credit_lines,...,loan_amount,term,interest_rate,installment,balance,paid_total,paid_principal,paid_interest,paid_late_fees,loan_status_new
0,3.0,90000.0,18.01,127914.571244,19.979304,0,38.000000,2001,6,28,...,28000,60,14.07,652.53,27015.86,1999.33,984.14,1015.19,0.0,0
1,10.0,40000.0,5.04,127914.571244,19.979304,0,36.760709,1996,1,30,...,5000,36,12.61,167.54,4651.37,499.12,348.63,150.49,0.0,0
2,3.0,40000.0,21.15,127914.571244,19.979304,0,28.000000,2006,4,31,...,2000,36,17.09,71.40,1824.63,281.80,175.37,106.43,0.0,0
3,1.0,30000.0,10.16,127914.571244,19.979304,0,36.760709,2007,0,4,...,21600,36,6.72,664.19,18853.26,3312.89,2746.74,566.15,0.0,0
4,10.0,35000.0,57.96,57000.000000,37.660000,0,36.760709,2008,7,22,...,23000,36,14.07,786.87,21430.15,2324.65,1569.85,754.80,0.0,0


In [56]:
inc_loan_no_int = logit('loan_status_new ~ debt_to_income + emp_length', data=df).fit()
print(inc_loan_no_int.params)

Optimization terminated successfully.
         Current function value: 0.057306
         Iterations 9
Intercept        -4.047437
debt_to_income   -0.019098
emp_length       -0.030737
dtype: float64


In [57]:
inc_loan_int = logit('loan_status_new ~ debt_to_income * emp_length', data=df).fit()
print(inc_loan_int.params)

Optimization terminated successfully.
         Current function value: 0.057303
         Iterations 9
Intercept                   -3.984926
debt_to_income              -0.022969
emp_length                  -0.042393
debt_to_income:emp_length    0.000702
dtype: float64


In [89]:
X = df.drop('loan_status_new', axis=1)
y = df['loan_status_new']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, 
                                                    random_state= 42, 
                                                    stratify= y)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [90]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 42 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   emp_length                        10000 non-null  float64
 1   annual_income                     10000 non-null  float64
 2   debt_to_income                    10000 non-null  float64
 3   annual_income_joint               10000 non-null  float64
 4   debt_to_income_joint              10000 non-null  float64
 5   delinq_2y                         10000 non-null  int64  
 6   months_since_last_delinq          10000 non-null  float64
 7   earliest_credit_line              10000 non-null  int64  
 8   inquiries_last_12m                10000 non-null  int64  
 9   total_credit_lines                10000 non-null  int64  
 10  open_credit_lines                 10000 non-null  int64  
 11  total_credit_limit                10000 non-null  int64  
 12  total

In [91]:
regression = LogisticRegression(random_state=42).fit(X_train, y_train)
y_predicted = regression.predict(X_test)

In [92]:
y_predicted

array([0, 0, 0, ..., 0, 0, 0])

In [93]:
regression.score(X_test, y_test)

0.9895